# 預測目標

預測六週後一週內的原物料需求量，以作為倉儲調度參考。

In [ ]:
import os

import pandas as pd  # python community standard 
import numpy as np

os.chdir(os.path.dirname(os.getcwd()))

from main.utils.utils import time_series_train_test_split, data_normalization, data_preparation
from main.model.LSTM_model import lstm_basic_model
from main.utils.data import load_data

df_parsed = load_data()

In [ ]:
import matplotlib.pyplot as plt

df_A = df_parsed[['A']]
df_A = df_A.resample('1D').sum()
df_A.fillna(0, inplace=True)

df_A['week'] = [idx.week for idx in df_A.index]
df_A['year'] = [idx.year for idx in df_A.index]

df_A_year_week = df_A.groupby(['week', 'year'], as_index=False)['A'].sum()

df_A_year_week.sort_values(by=['year', 'week'], inplace=True)

df_A_year_week.reset_index(drop=True, inplace=True)

fig, ax = plt.subplots(figsize=(16, 8))

ax.plot(df_A_year_week['A'])

In [ ]:
mean = df_A_year_week['A'].mean()
std = df_A_year_week['A'].std()

print(f"mean = {mean}, std = {std}")

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))

ax.plot(df_A_year_week['A'])
ax.axhline(y=mean, color='g', linestyle='-')
ax.axhline(y=mean + std, color='r', linestyle='-')
ax.axhline(y=mean - std, color='r', linestyle='-')

Strong week by week fluctuations. 
However, from business perspective, extremely accurate weekly consumption prediction is not that relevant. Because excessively ordered material can be left to the future. The two key goals are:

1. Make sure there is enough material for consumption.
2. Keep the storage reasonably low to reduce the warehouse cost.

Therefore, instead of predicting the exact weekly consumption, we predict a smoothed weekly consumption: 50% from the week and 50% from 4 weeks in the future average. 

In [ ]:
df_A_year_week['A'][:4].mean()

In [ ]:
data_size = len(df_A_year_week)

In [ ]:
four_weeks_average = [df_A_year_week['A'][i: min((i+4), data_size)].mean() for i in range(data_size)]

In [ ]:
df_A_year_week['A_four_weeks_average'] = four_weeks_average

In [ ]:
df_A_year_week['A_target'] = df_A_year_week[['A', 'A_four_weeks_average']].mean(axis=1)

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))

ax.plot(df_A_year_week['A'], c='b', label='raw data')
ax.plot(df_A_year_week['A_target'], c='r', label='processed data')

ax.legend(frameon=False)

In [ ]:
mean = df_A_year_week['A_target'].mean()
std = df_A_year_week['A_target'].std()

print(f"mean = {mean}, std = {std}")

In [ ]:
df_A_year_week['A_diff'] = df_A_year_week['A_target'] - df_A_year_week['A']
df_A_year_week['A_diff_cum'] = df_A_year_week['A_diff'].cumsum()

fig, ax = plt.subplots(figsize=(16, 8))

ax.plot(df_A_year_week['A_diff_cum'], c='b')
ax.plot(df_A_year_week['A_diff'], c='r')

# Data Engineering Pipeline

* history = 14
* future = 42
* duration = 7
* extension = 28

Then aggregate the day data to 7-days sum.

In [ ]:
frac = 0.8

history = 14
future = 42
duration = 7
extension = 28

df_train, df_test = time_series_train_test_split(df_A[['A']], frac=frac)
df_train_normalized, df_test_normalized = data_normalization(df_train, df_test)

X_train_days = np.array([df_train_normalized[i: i + history] for i in range(len(df_train_normalized) - history - future - extension)])

In [ ]:
X_train_days.shape

In [ ]:
weekly_data = [ X_train_days[:, i*7:(i+1)*7, :].sum(axis=1, keepdims=True) for i in range(history//7)]
X_train_weeks =  np.concatenate(weekly_data, axis=1)

In [ ]:
np.array(weekly_data).shape

In [ ]:
X_train_weeks.shape   # 962 rows, 2 weeks in the past, one feature

## Target engineering

In [ ]:
target = 'A'

y_train_days = np.array([df_train[target].values[i: i+extension] for i in range(history + future, len(df_train_normalized) - extension)])

In [ ]:
y_train_days.shape

In [ ]:
weekly_data = np.array([y_train_days[:, i*7:(i+1)*7].sum(axis=1, keepdims=True) for i in range(extension//7)])

In [ ]:
weekly_data.shape

In [ ]:
four_weeks_mean = weekly_data.mean(axis=0)
target_week = weekly_data[0, :, :]

fusion_data = np.concatenate([four_weeks_mean, target_week], axis=1).mean(axis=1)

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))

ax.plot(target_week, c='b')
ax.plot(fusion_data, c='r')

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

rescaled_fusion_data = scaler.fit_transform(fusion_data.reshape(-1, 1)).reshape(-1,)

## Let us write a pipeline line to make our life easier

In [ ]:
def feature_aggregation(df: pd.DataFrame, history: int, future: int, extension: int, duration: int):
    
    X_days = np.array([df[i: i + history] for i in range(len(df) - history - future - extension)])
    weekly_data = [ X_days[:, i*duration:(i+1)*duration, :].sum(axis=1, keepdims=True) for i in range(history//duration)]
    X_week =  np.concatenate(weekly_data, axis=1)
    
    return X_week


def target_aggregation(df: pd.DataFrame, history: int, future: int, extension: int, duration: int, target: str):
    
    y_days = np.array([df[target].values[i: i+extension] for i in range(history + future, len(df) - extension)])
    weekly_data = np.array([y_days[:, i*duration:(i+1)*duration].sum(axis=1, keepdims=True) for i in range(extension//duration)])
    
    four_weeks_mean = weekly_data.mean(axis=0)
    target_week = weekly_data[0, :, :]

    y_week = np.concatenate([four_weeks_mean, target_week], axis=1).mean(axis=1)
    
    return y_week


def data_transformation(df_train: pd.DataFrame, df_test: pd.DataFrame, history: int, future: int, extension: int, duration: int, target: str):
    
    df_train_normalized, df_test_normalized = data_normalization(df_train, df_test)
    
    X_train = feature_aggregation(df_train_normalized, history=history, future=future, extension=extension, duration=duration)
    X_test = feature_aggregation(df_test_normalized, history=history, future=future, extension=extension, duration=duration)
    
    assert (target in df_train.columns) and (target in df_test.columns)
    
    y_train = target_aggregation(df_train, history=history, future=future, extension=extension, duration=duration, target=target)
    y_test = target_aggregation(df_test, history=history, future=future, extension=extension, duration=duration, target=target)
    
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = data_transformation(df_train, df_test, history=history, future=future, extension=extension, duration=duration, target='A')

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
scaler = MinMaxScaler()

y_train = scaler.fit_transform(y_train.reshape(-1, 1)).reshape(-1,)
y_test = scaler.transform(y_test.reshape(-1, 1)).reshape(-1, )

## Simple Model

In [ ]:
from tensorflow.keras import optimizers

history_points = 2
features = 1

model = lstm_basic_model(history_points=history_points, features=features)
model.compile(optimizer=optimizers.Adam(lr = 0.001), loss='mse')
model.fit(x=X_train, y=y_train, batch_size=4, epochs=5, shuffle=True, validation_data=(X_test, y_test))



In [ ]:
y_pred = model.predict(X_test)

fig, ax = plt.subplots(figsize=(8,6))

ax.plot(y_pred, c='b', label='LSTM prediction')
ax.plot(y_test, c='r', label='Ground_truth')

ax.set_title("Validation set")

ax.legend()

Try add one more external feature: product C

In [ ]:
df_train, df_test = time_series_train_test_split(df_parsed[['A', 'C']], frac=frac)
X_train, X_test, y_train, y_test = data_transformation(df_train, df_test, history=history, future=future, extension=extension, duration=duration, target='A')
scaler = MinMaxScaler()

y_train = scaler.fit_transform(y_train.reshape(-1, 1)).reshape(-1,)
y_test = scaler.transform(y_test.reshape(-1, 1)).reshape(-1, )

history_points = 2
features = 2

model = lstm_basic_model(history_points=history_points, features=features)
model.compile(optimizer=optimizers.Adam(lr = 0.001), loss='mse')
model.fit(x=X_train, y=y_train, batch_size=4, epochs=5, shuffle=True, validation_data=(X_test, y_test))

In [ ]:
y_pred = model.predict(X_test)

fig, ax = plt.subplots(figsize=(8,6))

ax.plot(y_pred, c='b', label='LSTM prediction')
ax.plot(y_test, c='r', label='Ground_truth')

ax.set_title("Validation set")

ax.legend()

In [ ]:
# What if we ake more history points? like four weeks

history = 28

df_train, df_test = time_series_train_test_split(df_parsed[['A']], frac=frac)
X_train, X_test, y_train, y_test = data_transformation(df_train, df_test, history=history, future=future, extension=extension, duration=duration, target='A')
scaler = MinMaxScaler()

y_train = scaler.fit_transform(y_train.reshape(-1, 1)).reshape(-1,)
y_test = scaler.transform(y_test.reshape(-1, 1)).reshape(-1, )

history_points = 4
features = 1

model = lstm_basic_model(history_points=history_points, features=features)
model.compile(optimizer=optimizers.Adam(lr = 0.001), loss='mse')
model.fit(x=X_train, y=y_train, batch_size=4, epochs=5, shuffle=True, validation_data=(X_test, y_test))

In [ ]:
history = 21

df_train, df_test = time_series_train_test_split(df_parsed[['A']], frac=frac)
X_train, X_test, y_train, y_test = data_transformation(df_train, df_test, history=history, future=future, extension=extension, duration=duration, target='A')
scaler = MinMaxScaler()

y_train = scaler.fit_transform(y_train.reshape(-1, 1)).reshape(-1,)
y_test = scaler.transform(y_test.reshape(-1, 1)).reshape(-1, )

history_points = 3
features = 1

model = lstm_basic_model(history_points=history_points, features=features)
model.compile(optimizer=optimizers.Adam(lr = 0.001), loss='mse')
model.fit(x=X_train, y=y_train, batch_size=4, epochs=5, shuffle=True, validation_data=(X_test, y_test))

The model captures something but not response fast enough to the short time movement. Let us see if we can improve the model.

In [ ]:
def MACD(df: pd.DataFrame, period_fast, period_slow, signal, column: str, adjust: bool = True):
    
    """
    MACD, MACD Signal and MACD difference.
    The MACD Line oscillates above and below the zero line, which is also known as the centerline.
    These crossovers signal that the 12-day EMA has crossed the 26-day EMA. The direction, of course, depends on the direction of the moving average cross.
    Positive MACD indicates that the 12-day EMA is above the 26-day EMA. Positive values increase as the shorter EMA diverges further from the longer EMA.
    This means upside momentum is increasing. Negative MACD values indicates that the 12-day EMA is below the 26-day EMA.
    Negative values increase as the shorter EMA diverges further below the longer EMA. This means downside momentum is increasing.
    Signal line crossovers are the most common MACD signals. The signal line is a 9-day EMA of the MACD Line.
    As a moving average of the indicator, it trails the MACD and makes it easier to spot MACD turns.
    A bullish crossover occurs when the MACD turns up and crosses above the signal line.
    A bearish crossover occurs when the MACD turns down and crosses below the signal line.
    """
    
    EMA_fast = pd.Series(
            df[column].ewm(ignore_na=False, span=period_fast, adjust=adjust).mean(),
            name="EMA_fast",
        )
    EMA_slow = pd.Series(
        df[column].ewm(ignore_na=False, span=period_slow, adjust=adjust).mean(),
        name="EMA_slow",
    )
    MACD = pd.Series(EMA_fast - EMA_slow, name=f"{column}_MACD")
    MACD_signal = pd.Series(
        MACD.ewm(ignore_na=False, span=signal, adjust=adjust).mean(), name=f"{column}_SIGNAL"
    )

    return pd.concat([MACD, MACD_signal], axis=1)

In [ ]:
df = df_parsed[['A']]

In [ ]:
df_MACD = MACD(df,  period_fast=12, period_slow=26, signal=9, column='A')
df = pd.concat([df, df_MACD], axis=1)

In [ ]:
df.head(5)

In [ ]:
history=21

df_train, df_test = time_series_train_test_split(df, frac=frac)
X_train, X_test, y_train, y_test = data_transformation(df_train, df_test, history=history, future=future, extension=extension, duration=duration, target='A')
scaler = MinMaxScaler()

y_train = scaler.fit_transform(y_train.reshape(-1, 1)).reshape(-1,)
y_test = scaler.transform(y_test.reshape(-1, 1)).reshape(-1, )

history_points = 3
features = 3

model = lstm_basic_model(history_points=history_points, features=features)
model.compile(optimizer=optimizers.Adam(lr = 0.001), loss='mse')
model.fit(x=X_train, y=y_train, batch_size=4, epochs=5, shuffle=True, validation_data=(X_test, y_test))

In [ ]:
df = df_parsed[['A', 'C']]

df = df.resample('1D').sum()
df.fillna(0, inplace=True)

df_MACD = MACD(df,  period_fast=12, period_slow=26, signal=9, column='A')
df = pd.concat([df, df_MACD], axis=1)
# df_MACD = MACD(df,  period_fast=12, period_slow=26, signal=9, column='C')
# df = pd.concat([df, df_MACD], axis=1)

history=21

df_train, df_test = time_series_train_test_split(df, frac=frac)
X_train, X_test, y_train, y_test = data_transformation(df_train, df_test, history=history, future=future, extension=extension, duration=duration, target='A')
scaler = MinMaxScaler()

y_train = scaler.fit_transform(y_train.reshape(-1, 1)).reshape(-1,)
y_test = scaler.transform(y_test.reshape(-1, 1)).reshape(-1, )

history_points = 3
features = 4

model = lstm_basic_model(history_points=history_points, features=features)
model.compile(optimizer=optimizers.Adam(lr = 0.001), loss='mse')
model.fit(x=X_train, y=y_train, batch_size=4, epochs=5, shuffle=True, validation_data=(X_test, y_test))

In [ ]:
df = df_parsed[['A', 'C']]

df = df.resample('1D').sum()
df.fillna(0, inplace=True)

df_MACD = MACD(df,  period_fast=12, period_slow=26, signal=9, column='A')
df = pd.concat([df, df_MACD], axis=1)
df_MACD = MACD(df,  period_fast=12, period_slow=26, signal=9, column='C')
df = pd.concat([df, df_MACD], axis=1)

history=21

df_train, df_test = time_series_train_test_split(df, frac=frac)
X_train, X_test, y_train, y_test = data_transformation(df_train, df_test, history=history, future=future, extension=extension, duration=duration, target='A')
scaler = MinMaxScaler()

y_train = scaler.fit_transform(y_train.reshape(-1, 1)).reshape(-1,)
y_test = scaler.transform(y_test.reshape(-1, 1)).reshape(-1, )

history_points = 3
features = 6

model = lstm_basic_model(history_points=history_points, features=features)
model.compile(optimizer=optimizers.Adam(lr = 0.001), loss='mse')
model.fit(x=X_train, y=y_train, batch_size=4, epochs=5, shuffle=True, validation_data=(X_test, y_test))

In [ ]:
df_parsed.count()

In [ ]:
df = df_parsed[['A', 'C', 'G']]

df = df.resample('1D').sum()
df.fillna(0, inplace=True)

df_MACD = MACD(df,  period_fast=12, period_slow=26, signal=9, column='A')
df = pd.concat([df, df_MACD], axis=1)
df_MACD = MACD(df,  period_fast=12, period_slow=26, signal=9, column='C')
df = pd.concat([df, df_MACD], axis=1)
df_MACD = MACD(df,  period_fast=12, period_slow=26, signal=9, column='G')
df = pd.concat([df, df_MACD], axis=1)

history=21

df_train, df_test = time_series_train_test_split(df, frac=frac)
X_train, X_test, y_train, y_test = data_transformation(df_train, df_test, history=history, future=future, extension=extension, duration=duration, target='A')
scaler = MinMaxScaler()

y_train = scaler.fit_transform(y_train.reshape(-1, 1)).reshape(-1,)
y_test = scaler.transform(y_test.reshape(-1, 1)).reshape(-1, )

history_points = 3
features = 9

model = lstm_basic_model(history_points=history_points, features=features)
model.compile(optimizer=optimizers.Adam(lr = 0.001), loss='mse')
model.fit(x=X_train, y=y_train, batch_size=4, epochs=5, shuffle=True, validation_data=(X_test, y_test))